<a href="https://colab.research.google.com/github/Pihhot/SpeechType_Advanced/blob/main/speech_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inputs for NN

In [42]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import to_categorical, pad_sequences

from sklearn.preprocessing import LabelEncoder
import numpy as np

from keras.layers import Dense, LSTM, Input, Dropout, Embedding, Concatenate, Input, GRU, Bidirectional
from keras.models import Sequential, Model
from keras.optimizers import Adam

Create Model

In [68]:
input_1 = Input(shape=(maxlen,), name='sentence')
embedding_1 = Embedding(maxWordsCount, 100, input_length=maxlen)(input_1)


input_2 = Input(shape=(maxlen,), name='deps')
embedding_2 = Embedding(maxWordsCount, 100, input_length=maxlen)(input_2)


input_3 = Input(shape=(maxlen,), name='poss')
embedding_3 = Embedding(maxWordsCount, 100, input_length=maxlen)(input_3)


concat = Concatenate()([embedding_1, embedding_2, embedding_3])

lstm_1 = LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)(concat)

lstm_2 = LSTM(64, dropout=0.1, recurrent_dropout=0.1, return_sequences=False)(lstm_1)

dense = Dense(43, activation='softmax')(lstm_2)

model = Model(inputs=[input_1, input_2, input_3], outputs=dense)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.001))

Fit model

In [ ]:
history = model.fit([sentences_m, deps, poss], classes, batch_size=32, epochs=50, validation_data=([sentences_m_t, deps_t, poss_t], classes_t), shuffle=True)

Epoch 1/50
625/625 [==============================] - 44s 63ms/step - loss: 1.7511 - accuracy: 0.5134 - val_loss: 1.5679 - val_accuracy: 0.5540
Epoch 2/50
625/625 [==============================] - 38s 61ms/step - loss: 1.4919 - accuracy: 0.5600 - val_loss: 1.3799 - val_accuracy: 0.5870
Epoch 3/50
625/625 [==============================] - 38s 60ms/step - loss: 1.3558 - accuracy: 0.5895 - val_loss: 1.2543 - val_accuracy: 0.6187
Epoch 4/50
625/625 [==============================] - 38s 60ms/step - loss: 1.2620 - accuracy: 0.6143 - val_loss: 1.1764 - val_accuracy: 0.6351
Epoch 5/50
625/625 [==============================] - 38s 61ms/step - loss: 1.1981 - accuracy: 0.6318 - val_loss: 1.1159 - val_accuracy: 0.6631
Epoch 6/50
625/625 [==============================] - 38s 61ms/step - loss: 1.1398 - accuracy: 0.6480 - val_loss: 1.0525 - val_accuracy: 0.6813
Epoch 7/50
625/625 [==============================] - 37s 59ms/step - loss: 1.0857 - accuracy: 0.6636 - val_loss: 0.9928 - val_accuracy:

Dowload and prepare predprocessed data for NN

In [53]:
import pandas as pd
from pathlib import Path

# Dowload data from git
#!wget https://raw.githubusercontent.com/Pihhot/SpeechType_Advanced/main/train_data
#!wget https://raw.githubusercontent.com/Pihhot/SpeechType_Advanced/main/test_data

train_df_path = Path.cwd() / 'train_data'
train_df = pd.read_csv(str(train_df_path), sep=':')

test_df_path = Path.cwd() / 'train_data'
test_df = pd.read_csv(str(train_df_path), sep=':')

sentences_m = list(train_df['MODIFIED'])
deps = list(train_df['DEPS'])
poss = list(train_df['POSS'])
classes = list(train_df['TYPE'])

sentences_m_t = list(test_df['MODIFIED'])
deps_t = list(test_df['DEPS'])
poss_t = list(test_df['POSS'])
classes_t = list(test_df['TYPE'])


# Train Tokenizer
maxWordsCount = 4000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(sentences_m+deps+poss+sentences_m_t)
# print(list(tokenizer.word_counts.items())[:10])

# Prepare text information
def texts_to_s(texts: list):
  return tokenizer.texts_to_sequences(texts)

sentences_m, deps, poss = texts_to_s(sentences_m), texts_to_s(deps), texts_to_s(poss)
sentences_m_t, deps_t, poss_t = texts_to_s(sentences_m_t), texts_to_s(deps_t), texts_to_s(poss_t)

maxlen = 10
sentences_m, deps, poss = pad_sequences(sentences_m, maxlen=maxlen), pad_sequences(deps, maxlen=maxlen), pad_sequences(poss, maxlen=maxlen)
sentences_m_t, deps_t, poss_t = pad_sequences(sentences_m_t, maxlen=maxlen), pad_sequences(deps_t, maxlen=maxlen), pad_sequences(poss_t, maxlen=maxlen)

# Prepare classes
classes = np.array(classes)
classes_t = np.array(classes)

l_encoder = LabelEncoder()

classes = l_encoder.fit_transform(classes)
classes = to_categorical(classes)

classes_t = l_encoder.fit_transform(classes_t)
classes_t = to_categorical(classes_t)



*   Download RAW data from git
*   Process RAW data
*   Split on train & test





In [ ]:
import pandas as pd
import spacy
!pip install contractions
import contractions
from pathlib import Path
import json
import re
#####################################################################################
!wget https://raw.githubusercontent.com/Pihhot/SpeechType_Advanced/main/data.json

data_path = Path.cwd() / 'data.json'

with open(data_path) as f:
    data = json.load(f)
######################################################################################    

def sent_lemmas(doc) -> str:
    return ' '.join(t.lemma_ for t in doc)

def sent_deps(doc) -> str:
    return ' '.join(t.dep_ for t in doc)

def sent_poss(doc) -> str:
    return ' '.join(t.pos_ for t in doc)

def clear_sentence(text: str) -> str:
    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    # Removing multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text.lower()

def expand_contractions(text: str) -> str:
    return ' '.join([contractions.fix(w) for w in text.split(' ')])

data_c = data.copy()

nlp = spacy.load("en_core_web_sm")


train_sentences_or, train_sentences_m, train_deps, train_pos, train_class = [], [], [], [], []
test_sentences_or, test_sentences_m, test_deps, test_pos, test_class = [], [], [], [], []


index = 0
for class_ in data.keys():
  for sentence in data[class_]:
    index += 1
    # Save original sentences
    sentence_or = sentence
    # Beatify sentence
    sentence = expand_contractions(sentence)
    sentence = clear_sentence(sentence)
    if len(sentence):
      # Sentence representations
      doc = nlp(sentence)
      deps_str = sent_deps(doc)
      poss_str = sent_poss(doc)
      lemmas_str = sent_lemmas(doc)
      # Add every fifth element to test
      if index % 7 == 0:
        train_sentences_or.append(sentence_or)
        train_sentences_m.append(lemmas_str)
        train_deps.append(deps_str)
        train_pos.append(poss_str)
        train_class.append(class_)
      else:
        test_sentences_or.append(sentence_or)
        test_sentences_m.append(lemmas_str)
        test_deps.append(deps_str)
        test_pos.append(poss_str)
        test_class.append(class_)



train_df_data = {'ORIGIN':train_sentences_or,
                 'MODIFIED':train_sentences_m,
                 'DEPS':train_deps,
                 'POSS':train_pos,
                 'TYPE':train_class}

test_df_data = {'ORIGIN':test_sentences_or,
                 'MODIFIED':test_sentences_m,
                 'DEPS':test_deps,
                 'POSS':test_pos,
                 'TYPE':test_class}

train_df = pd.DataFrame(train_df_data)
test_df = pd.DataFrame(test_df_data)

test_df.to_csv('test_data.csv', index=False, sep=':')
train_df.to_csv('train_data.csv', index=False, sep=':')